In [50]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2



In [51]:
df = pd.read_csv('universities-clean.csv')

In [63]:


try:
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        user="amdariuser",
        password="amdariuserpassword",
        database="amdaridb"
    )
    print("✅ Connected to local PostgreSQL successfully!")
    
except Exception as e:
    print("❌ Failed to connect:", e)


✅ Connected to local PostgreSQL successfully!


In [65]:
cursor = conn.cursor()

In [59]:
create_query_file = open('./sql/postgres_create_table.sql')
create_query = create_query_file.read()
create_query

'\n-- Create tables for university ranking database in PostgreSQL\n-- Database: university_ranking\n\n-- Drop tables if they exist (in reverse order to avoid FK conflicts)\nDROP TABLE IF EXISTS InstitutionRanking;\nDROP TABLE IF EXISTS Institution;\nDROP TABLE IF EXISTS Location;\nDROP TABLE IF EXISTS Country;\nDROP TABLE IF EXISTS Continent;\nDROP TABLE IF EXISTS Affiliation;\nDROP TABLE IF EXISTS DeliveryMode;\n\n-- Create Continent table\nCREATE TABLE Continent (\n    ContinentID VARCHAR(4) PRIMARY KEY,\n    ContinentName VARCHAR(50) NOT NULL UNIQUE\n);\n\n-- Create Country table\nCREATE TABLE Country (\n    CountryID VARCHAR(4) PRIMARY KEY,\n    CountryName VARCHAR(100) NOT NULL UNIQUE,\n    ContinentID VARCHAR(4) NOT NULL,\n    CONSTRAINT fk_country_continent FOREIGN KEY (ContinentID) REFERENCES Continent(ContinentID)\n);\n\n-- Create Location table\nCREATE TABLE Location (\n    LocationID VARCHAR(4) PRIMARY KEY,\n    City VARCHAR(100) NOT NULL,\n    CountryID VARCHAR(4) NOT NULL,

In [60]:
cursor.execute(create_query)

In [61]:
conn.commit()

In [42]:
engine = create_engine("postgresql+psycopg2://", creator=lambda: conn)

In [43]:
pd.read_sql("SELECT * FROM continent;", con=engine)

,continentid,continentname


In [89]:
# 1. Read and parse the upsert SQL file by tags
with open('./sql/postgres_upsert.sql') as f:
    sql_blocks = f.read().split('--')
    upsert_queries = {}
    for block in sql_blocks:
        lines = block.strip().splitlines()
        if lines:
            key = lines[0].strip().replace(" ", "_").upper()  # normalize tag names
            query = '\n'.join(lines[1:])
            upsert_queries[key] = query

# 2. Loop through df and insert per table
for row in df.itertuples(index=False):
    cursor.execute(upsert_queries['CONTINENT'], (row.ContinentID, row.Continent))
for row in df.itertuples(index=False):
    cursor.execute(upsert_queries['COUNTRY'], (row.CountryID, row.Country, row.ContinentID))
for row in df.itertuples(index=False):
    cursor.execute(upsert_queries['LOCATION'], (row.LocationID, row.city, row.CountryID))
for row in df.itertuples(index=False):
    cursor.execute(upsert_queries['AFFILIATION'], (row.AffiliationID, row.Affiliation))
for row in df.itertuples(index=False):
    cursor.execute(upsert_queries['DELIVERYMODE'], (row.DeliveryModeID, row.DeliveryMode))
for row in df.itertuples(index=False):
    cursor.execute(upsert_queries['INSTITUTION'], (
        row.InstitutionID, row.Institution, row.Founded,
        row.link, row.LocationID
    ))
for row in df.itertuples(index=False):
    cursor.execute(upsert_queries['INSTITUTIONRANKING'], (
        row.RankingID, row.InstitutionID, row.LocationID, row.Enrollment,
        row.AffiliationID, row.DeliveryModeID, row.Rank
    ))

# 3. Commit the transaction
conn.commit()
